In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from functools import partial
from multiprocessing import shared_memory
from multiprocessing.dummy import Pool
from sklearn.ensemble import RandomForestRegressor
import multiprocessing as mp
from itertools import chain, combinations
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel

from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import sys
import os

import mp_run

import concurrent.futures
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

In [2]:
perturbation_factor = 3
num_rf_predictors = 500

target_tf = 'AT2G46680'

induction_flag = 1
mp_threads = 20
# if (len(sys.argv)>=3):
#     induction_flag = bool(sys.argv[1])
#     mp_threads = int(sys.argv[2])

In [3]:
nof_iters = 10000

def pairedtest(n1, n2, nof_iters):
    import random
   
    ndiff = sum([ n1[i]-n2[i] for i in range(len(n1)) ])
    if ndiff < 0:
         nbig = n2
         nsmall = n1
         ndiff = -ndiff
    else:
         nbig = n1
         nsmall = n2
    
    bcount = 0
    for niter in range(nof_iters):
        tdiff = 0
        for i in range(len(nbig)):
            a = random.random()
            if a <0.5:
                tdiff += nsmall[i]-nbig[i]
            else:
                tdiff += nbig[i]-nsmall[i]
        if tdiff > ndiff:
            bcount += 1
    return bcount / nof_iters

In [4]:
tf_df = pd.read_csv('data/wrky_regulators.csv')
tf_list = tf_df['Gene']
scaler = StandardScaler()

In [5]:
def choose_2_3(iterable):
    "powerset([1,2,3]) -->  (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(2,4))

In [6]:
target_df = pd.read_csv('data/wrky_targets_neg.csv')
if (induction_flag):
    target_df = pd.read_csv('data/wrky_targets_pos.csv')
deg_genes = target_df['Gene']

In [7]:
ts_df_raw = pd.read_csv('data/GSE97500/expression.tsv', sep='\t', index_col=0)
ts_df = pd.DataFrame(scaler.fit_transform(ts_df_raw), columns=ts_df_raw.columns)
ts_df.index = ts_df_raw.index

c = pd.Series(list(set(deg_genes).intersection(set(ts_df.index))))
tf_list = pd.Series(list(set(tf_list).intersection(set(ts_df.index))))
meta_df = pd.read_csv('data/GSE97500/meta_data.tsv', sep='\t')
ts_exp_index = meta_df[meta_df['isTs']]
ts_exp_index_target =  ts_exp_index[ts_exp_index['is1stLast'] == 'm'].condName
ts_exp_index_source =  ts_exp_index[ts_exp_index['is1stLast'] == 'm'].prevCol

In [8]:
ts_exp_index_target_val = ts_exp_index[ts_exp_index['is1stLast'] == 'l'].condName
ts_exp_index_source_val =  ts_exp_index[ts_exp_index['is1stLast'] == 'l'].prevCol

In [9]:
target_genes = set(deg_genes).intersection(set(ts_df.index))

In [10]:
non_trivial_targets = []
for target in target_genes:
    if ts_df.loc[target].mean() != 0.0:
        non_trivial_targets.append(target)
target_genes = pd.Series(non_trivial_targets)

In [16]:
# efron trick #1
regr = RandomForestRegressor(random_state=0)
# regr = xgb.XGBRegressor()

ts_train_y_list = ts_df[ts_exp_index_target]
# set training target to expression level change
ts_train_y_list.iloc[:, :] = ts_df[ts_exp_index_target].values - ts_df[ts_exp_index_source].values
ts_val_y_list = ts_df[ts_exp_index_target_val]
# set training target to expression level change
ts_val_y_list.iloc[:, :] = ts_df[ts_exp_index_target_val].values - ts_df[ts_exp_index_source_val].values

ts_val_naive_y_list = ts_df[ts_exp_index_source_val]
ts_val_naive_y_list.loc[:,:] = 0


result_list = []
result_measure_list = []
mse_list = []
naive_mse_list = []
bottom_mse_list = []
top_feature_size_list = []


for target_gene in tqdm(target_genes):
    train_gene_index = tf_list[tf_list != target_gene]
    ts_val_y = ts_val_y_list.loc[target_gene]
    old_mse = sys.maxsize
    old_mse = np.random.randn(len(ts_val_y)) + sys.maxsize
    top_influence_genes = train_gene_index
    data_mean = ts_df.T[top_influence_genes].mean()
    data_std = ts_df.T[top_influence_genes].std()
    ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
    ts_val_X = ts_df[ts_exp_index_source_val].T[top_influence_genes]
    ts_train_y = ts_train_y_list.loc[target_gene]
    ts_val_y = ts_val_y_list.loc[target_gene]
    ts_val_naive_y = ts_val_naive_y_list.loc[target_gene]
    regr = RandomForestRegressor(random_state=42, warm_start=True, n_estimators=300, n_jobs=20)
    regr = regr.fit(ts_train_X, ts_train_y)
    # cur_mse = mean_squared_error(ts_val_y,regr.predict(ts_val_X))
    cur_mse = np.square(ts_val_y - regr.predict(ts_val_X))
    old_mse = cur_mse
    first_mse = cur_mse
    top_half = np.argsort(regr.feature_importances_)[int(top_influence_genes.size/2):]
    old_top_influence_genes = top_influence_genes
    top_influence_genes = top_influence_genes.iloc[top_half]
    if (target_tf not in top_influence_genes.values):
        top_influence_genes = top_influence_genes.append(tf_list[tf_list == target_tf])
    old_data_mean = data_mean
    old_data_std = data_std
    old_regr = regr
    old_ts_val_X = ts_val_X


    while(True):
        data_mean = ts_df.T[top_influence_genes].mean()
        data_std = ts_df.T[top_influence_genes].std()
        regr = RandomForestRegressor(random_state=42, warm_start=True, n_estimators=300, n_jobs=20)
        ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
        ts_val_X = ts_df[ts_exp_index_source_val].T[top_influence_genes]
        ts_train_y = ts_train_y_list.loc[target_gene]
        ts_val_y = ts_val_y_list.loc[target_gene]
        ts_val_naive_y = ts_val_naive_y_list.loc[target_gene]
        regr = regr.fit(ts_train_X, ts_train_y)
        # cur_mse = mean_squared_error(ts_val_y,regr.predict(ts_val_X))
        cur_mse = np.square(ts_val_y - regr.predict(ts_val_X))
        p = pairedtest(cur_mse, first_mse, 10000)
        if (p < 0.05 and np.mean(cur_mse) > np.mean(first_mse)) or len(top_influence_genes) < 3:
            data_mean = old_data_mean
            data_std = old_data_std
            regr = old_regr
            ts_val_X = old_ts_val_X
            top_influence_genes = old_top_influence_genes
            break
        else:
            old_mse = cur_mse
            top_half = np.argsort(regr.feature_importances_)[int(top_influence_genes.size/2):]
            old_top_influence_genes = top_influence_genes
            top_influence_genes = top_influence_genes.iloc[top_half]
            if (target_tf not in top_influence_genes.values):
                top_influence_genes = top_influence_genes.append(tf_list[tf_list == target_tf])
            # if (len(old_top_influence_genes) == len(top_influence_genes)):
            if (len(old_top_influence_genes) == len(top_influence_genes) or len(top_influence_genes) < 3):
                data_mean = old_data_mean
                data_std = old_data_std
                regr = old_regr
                ts_val_X = old_ts_val_X
                top_influence_genes = old_top_influence_genes
                break

            old_data_mean = data_mean
            old_data_std = data_std
            old_regr = regr
            old_ts_val_X = ts_val_X

            
    top_feature_size_list.append(top_influence_genes.size)
    if (top_influence_genes.size < tf_list.size):
        bottom_influence_genes = list(set(tf_list) - (set(top_influence_genes)))
        bottom_regr = RandomForestRegressor(random_state=42, warm_start=True, n_estimators=300, n_jobs=20)
        ts_train_X_comp = ts_df[ts_exp_index_source].T[bottom_influence_genes]
        ts_val_X_comp = ts_df[ts_exp_index_source_val].T[bottom_influence_genes]
        ts_train_y = ts_train_y_list.loc[target_gene]
        ts_val_y = ts_val_y_list.loc[target_gene]
        bottom_regr = bottom_regr.fit(ts_train_X_comp, ts_train_y)
        bottom_mse_list.append(mean_squared_error(ts_val_y,bottom_regr.predict(ts_val_X_comp)))
    else:
        bottom_mse_list.append(0)

    

    input_mean = data_mean
    # input_std = ts_train_X.std()
    naive_mse_list.append(mean_squared_error(ts_val_y,ts_val_naive_y))
    mse_list.append(mean_squared_error(ts_val_y,regr.predict(ts_val_X)))

    base_prediction = regr.predict(np.array(input_mean).reshape(1,-1))[0]
    y_std = ts_df.T.std()[target_gene]
    perturbation_list = list(choose_2_3(top_influence_genes))
    # perturbation_list = [set(perturbation).union(set([target_tf])) for perturbation in perturbation_list]
    # perturbation_list.insert(0, set([target_tff]))
    
    perturbation_input_list = []
    for perturbation_genes in perturbation_list:
        perturbation_input = data_mean.copy()
        for gene in perturbation_genes:
            perturbation_input[gene] += data_std[gene] * perturbation_factor
        perturbation_input_list.append(perturbation_input)
    
    perturbation_list_names = ['; '.join(perturbation_genes) for perturbation_genes in perturbation_list]
    perturbation_input = np.array(perturbation_input_list)
    perturbation_result_list = (regr.predict(perturbation_input) - base_prediction)/y_std
    if (perturbation_result_list[0] > 0):
        result_list.append(np.array(perturbation_list_names)[np.argsort(perturbation_result_list)[::-1][:5]])
        result_measure_list.append(np.array(perturbation_result_list)[np.argsort(perturbation_result_list)[::-1][:5]])
    else:
        result_list.append(np.array(perturbation_list_names)[np.argsort(perturbation_result_list)[:5]])
        result_measure_list.append(np.array(perturbation_result_list)[np.argsort(perturbation_result_list)[:5]])
   
    result_list.append([target_tf])
    result_measure_list.append(perturbation_result_list[0])


100%|██████████| 775/775 [1:16:48<00:00,  5.95s/it]


In [11]:
# efron trick #2
regr = RandomForestRegressor(random_state=0)
# regr = xgb.XGBRegressor()

ts_train_y_list = ts_df[ts_exp_index_target]
# set training target to expression level change
ts_train_y_list.iloc[:, :] = ts_df[ts_exp_index_target].values - ts_df[ts_exp_index_source].values
ts_val_y_list = ts_df[ts_exp_index_target_val]
# set training target to expression level change
ts_val_y_list.iloc[:, :] = ts_df[ts_exp_index_target_val].values - ts_df[ts_exp_index_source_val].values

ts_val_naive_y_list = ts_df[ts_exp_index_source_val]
ts_val_naive_y_list.loc[:,:] = 0


result_list = []
result_measure_list = []
mse_list = []
naive_mse_list = []
bottom_mse_list = []
top_feature_size_list = []


for target_gene in tqdm(target_genes):
    train_gene_index = tf_list[tf_list != target_gene]
    ts_val_y = ts_val_y_list.loc[target_gene]
    old_mse = sys.maxsize
    old_mse = np.random.randn(len(ts_val_y)) + sys.maxsize
    top_influence_genes = train_gene_index
    data_mean = ts_df.T[top_influence_genes].mean()
    data_std = ts_df.T[top_influence_genes].std()
    ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
    ts_val_X = ts_df[ts_exp_index_source_val].T[top_influence_genes]
    ts_train_y = ts_train_y_list.loc[target_gene]
    ts_val_y = ts_val_y_list.loc[target_gene]
    ts_val_naive_y = ts_val_naive_y_list.loc[target_gene]
    regr = RandomForestRegressor(random_state=42, warm_start=False, n_estimators=300, n_jobs=20)
    regr = regr.fit(ts_train_X, ts_train_y)
    # cur_mse = mean_squared_error(ts_val_y,regr.predict(ts_val_X))
    cur_mse = np.square(ts_val_y - regr.predict(ts_val_X))
    old_mse = cur_mse
    first_mse = cur_mse
    outer_loop_flag = True
    feature_pool = train_gene_index
    selected_features = pd.Series([])
    while (outer_loop_flag):
        top_influence_genes = feature_pool
        ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
        ts_val_X = ts_df[ts_exp_index_source_val].T[top_influence_genes]
        regr = regr.fit(ts_train_X, ts_train_y)
        cur_mse = np.square(ts_val_y - regr.predict(ts_val_X))
        p = pairedtest(cur_mse, first_mse, 10000)
        if (p < 0.05 and np.mean(cur_mse) > np.mean(first_mse)) or len(top_influence_genes) < 3:
            outer_loop_flag = False
        top_half = np.argsort(regr.feature_importances_)[int(top_influence_genes.size/2):]
        old_top_influence_genes = top_influence_genes
        top_influence_genes = top_influence_genes.iloc[top_half]
        old_data_mean = data_mean
        old_data_std = data_std
        old_regr = regr
        old_ts_val_X = ts_val_X

        inner_loop_flag = True

        while(inner_loop_flag):
            data_mean = ts_df.T[top_influence_genes].mean()
            data_std = ts_df.T[top_influence_genes].std()
            regr = RandomForestRegressor(random_state=42, warm_start=False, n_estimators=300, n_jobs=20)
            ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
            ts_val_X = ts_df[ts_exp_index_source_val].T[top_influence_genes]
            ts_train_y = ts_train_y_list.loc[target_gene]
            ts_val_y = ts_val_y_list.loc[target_gene]
            ts_val_naive_y = ts_val_naive_y_list.loc[target_gene]
            regr = regr.fit(ts_train_X, ts_train_y)
            # cur_mse = mean_squared_error(ts_val_y,regr.predict(ts_val_X))
            cur_mse = np.square(ts_val_y - regr.predict(ts_val_X))
            p = pairedtest(cur_mse, first_mse, 10000)
            if (p < 0.05 and np.mean(cur_mse) > np.mean(first_mse)) or len(top_influence_genes) < 3:
                data_mean = old_data_mean
                data_std = old_data_std
                regr = old_regr
                ts_val_X = old_ts_val_X
                top_influence_genes = old_top_influence_genes
                inner_loop_flag = False
            else:
                old_mse = cur_mse
                top_half = np.argsort(regr.feature_importances_)[int(top_influence_genes.size/2):]
                old_top_influence_genes = top_influence_genes
                top_influence_genes = top_influence_genes.iloc[top_half]
                # if (len(old_top_influence_genes) == len(top_influence_genes)):
                if (len(old_top_influence_genes) == len(top_influence_genes) or len(top_influence_genes) < 3):
                    data_mean = old_data_mean
                    data_std = old_data_std
                    regr = old_regr
                    ts_val_X = old_ts_val_X
                    top_influence_genes = old_top_influence_genes
                    inner_loop_flag = False

                old_data_mean = data_mean
                old_data_std = data_std
                old_regr = regr
                old_ts_val_X = ts_val_X
        
        feature_pool = feature_pool[~feature_pool.isin(top_influence_genes)]
        if (feature_pool.size == 0):
            outer_loop_flag = False
        selected_features = selected_features.append(top_influence_genes)
        
        
        
                



    top_influence_genes = selected_features
    if (target_tf not in top_influence_genes.values):
        top_influence_genes = top_influence_genes.append(tf_list[tf_list == target_tf])
    
    data_mean = ts_df.T[top_influence_genes].mean()
    data_std = ts_df.T[top_influence_genes].std()
    ts_train_X = ts_df[ts_exp_index_source].T[top_influence_genes]
    ts_val_X = ts_df[ts_exp_index_source_val].T[top_influence_genes]
    input_mean = data_mean
    input_std = data_std

    regr = regr.fit(ts_train_X, ts_train_y)
    
    top_feature_size_list.append(top_influence_genes.size)
    if (top_influence_genes.size < tf_list.size):
        bottom_influence_genes = list(set(tf_list) - (set(top_influence_genes)))
        bottom_regr = RandomForestRegressor(random_state=42, warm_start=True, n_estimators=300, n_jobs=20)
        ts_train_X_comp = ts_df[ts_exp_index_source].T[bottom_influence_genes]
        ts_val_X_comp = ts_df[ts_exp_index_source_val].T[bottom_influence_genes]
        ts_train_y = ts_train_y_list.loc[target_gene]
        ts_val_y = ts_val_y_list.loc[target_gene]
        bottom_regr = bottom_regr.fit(ts_train_X_comp, ts_train_y)
        bottom_mse_list.append(mean_squared_error(ts_val_y,bottom_regr.predict(ts_val_X_comp)))
    else:
        bottom_mse_list.append(0)

    
    naive_mse_list.append(mean_squared_error(ts_val_y,ts_val_naive_y))
    mse_list.append(mean_squared_error(ts_val_y,regr.predict(ts_val_X)))

    base_prediction = regr.predict(np.array(input_mean).reshape(1,-1))[0]
    y_std = ts_df.T.std()[target_gene]
    perturbation_list = list(choose_2_3(top_influence_genes))
    # perturbation_list = [set(perturbation).union(set([target_tf])) for perturbation in perturbation_list]
    # perturbation_list.insert(0, set([target_tff]))
    
    perturbation_input_list = []
    for perturbation_genes in perturbation_list:
        perturbation_input = data_mean.copy()
        for gene in perturbation_genes:
            perturbation_input[gene] += data_std[gene] * perturbation_factor
        perturbation_input_list.append(perturbation_input)
    
    perturbation_list_names = ['; '.join(perturbation_genes) for perturbation_genes in perturbation_list]
    perturbation_input = np.array(perturbation_input_list)
    perturbation_result_list = (regr.predict(perturbation_input) - base_prediction)/y_std
    if (perturbation_result_list[0] > 0):
        result_list.append(np.array(perturbation_list_names)[np.argsort(perturbation_result_list)[::-1][:5]])
        result_measure_list.append(np.array(perturbation_result_list)[np.argsort(perturbation_result_list)[::-1][:5]])
    else:
        result_list.append(np.array(perturbation_list_names)[np.argsort(perturbation_result_list)[:5]])
        result_measure_list.append(np.array(perturbation_result_list)[np.argsort(perturbation_result_list)[:5]])
   
    result_list.append([target_tf])
    result_measure_list.append(perturbation_result_list[0])


  1%|          | 8/775 [08:37<14:40:04, 68.85s/it]

In [17]:
fixed_result_list = []
for i in range(0, len(result_list), 2):
    current_list = result_list[i]
    if (len(current_list) == 4):
        current_list = np.append(current_list, 'NA')
    fixed_result_list.append(np.append(current_list, target_tf))

In [18]:
fixed_result_measure_list = []
for i in range(0, len(result_measure_list), 2):
    current_list = result_measure_list[i]
    if (len(current_list) == 4):
        current_list = np.append(current_list, 0)
    fixed_result_measure_list.append(np.append(current_list, result_measure_list[i+1]))

In [19]:
fixed_result_measure_list = np.array(fixed_result_measure_list)
fixed_result_list = np.array(fixed_result_list)

In [22]:
fixed_result_measure_list.shape

(775, 6)

In [23]:
result_measure_list = np.array(result_measure_list)
result_list = np.array(result_list)
out_df = pd.DataFrame()
out_df.index = target_genes
for i in range(6):
    comb_name = 'top_{}_combination'.format(i+1)
    score_name = 'top_{}_score'.format(i+1)
    out_df[comb_name] = fixed_result_list[:,i]
    out_df[score_name] = fixed_result_measure_list[:,i]
out_df['mse'] = mse_list
out_df['naive_mse'] = naive_mse_list
out_df['mse_diff'] = np.array(mse_list) - np.array(naive_mse_list)
out_df['bottom_mse'] = bottom_mse_list

out_df['feature_size'] = top_feature_size_list

In [24]:
# out_df['top_comb'] = fixed_result_list
# out_df['top_comb_score'] = fixed_result_measure_list


In [24]:
(out_df[out_df['mse_diff'] > 0]['top_6_score'] > 0).sum()

303

In [25]:
(out_df['mse_diff'] > 0).sum() 

522

In [26]:
out_df.to_csv('output/wrky_presentation_pos_rf_new.csv')

In [3]:
out_df = pd.read_csv('output/wrky_presentation_pos_rf_new.csv', index_col=0)

In [5]:
import re
regex = re.compile(r'[-+]?\d*\.\d+|\d+')

res_list = []
for i in out_df['top_comb_score'].values:
    numbers = regex.findall(i)
    value = float(numbers[-1])
    res_list.append(value)
out_df['top_6_score'] = res_list
(out_df['top_6_score'] > 0).sum()

472

In [26]:
count = 0
for i in fixed_result_measure_list:
    if (i[-1] > 0):
        count += 1
print(count)

504


In [122]:
(out_df[out_df['mse_diff'] < 0]['top_6_score'] < 0).sum()

259

In [28]:
out_df_pos = pd.read_csv('./output/wrky_presentation_pos_new.csv', index_col=0)

In [55]:
(out_df_pos[out_df_pos['mse_diff'] < 0]['top_6_score'] < 0).sum()

331

In [56]:
out_df_pos[out_df_pos['mse_diff'] < 0]

,top_1_combination,top_1_score,top_2_combination,top_2_score,top_3_combination,top_3_score,top_4_combination,top_4_score,top_5_combination,top_5_score,top_6_combination,top_6_score,mse,naive_mse,mse_diff
AT4G27870,AT4G31550; AT2G40750; AT4G11070; AT2G46680,-0.241358,AT4G31550; AT5G01900; AT2G40750; AT2G46680,-0.241016,AT4G04450; AT4G11070; AT2G46680; AT4G31550,-0.237097,AT4G31550; AT5G01900; AT4G11070; AT2G46680,-0.235502,AT5G01900; AT4G11070; AT2G46680; AT2G40750,-0.235193,AT2G46680,-0.007325,2.709843e+06,3.182896e+06,-473052.990483
AT1G07200,AT5G15130; AT2G40740; AT2G46680; AT2G38470,0.100623,AT5G41570; AT2G40740; AT2G46680; AT2G38470,0.098149,AT5G41570; AT5G15130; AT2G46680; AT2G38470,0.093707,AT1G68150; AT2G46680; AT5G41570; AT2G38470,0.093002,AT1G68150; AT5G15130; AT2G46680; AT2G38470,0.090602,AT2G46680,0.000114,1.349837e+06,1.532306e+06,-182468.444314
AT1G21528,AT1G69810; AT4G26440; AT1G66560; AT2G46680,0.708753,AT4G26440; AT1G66560; AT2G34830; AT2G46680,0.602587,AT1G69810; AT1G66560; AT2G34830; AT2G46680,0.585824,AT4G26440; AT4G11070; AT1G66560; AT2G46680,0.585236,AT1G69810; AT4G11070; AT1G66560; AT2G46680,0.575825,AT2G46680,0.166748,4.541715e+02,5.343333e+02,-80.161868
AT5G14210,AT1G69310; AT2G25000; AT2G46680; AT5G45050,-0.357855,AT4G01250; AT1G69310; AT2G25000; AT2G46680,-0.356109,AT2G30590; AT1G69310; AT2G25000; AT2G46680,-0.345043,AT1G69310; AT2G25000; AT1G29860; AT2G46680,-0.343307,AT1G29280; AT1G69310; AT2G25000; AT2G46680,-0.338402,AT2G46680,-0.059871,1.295461e+06,1.424367e+06,-128905.893070
AT2G31790,AT1G29280; AT2G03340; AT2G25000; AT2G46680,-0.225702,AT1G29280; AT1G80590; AT5G46350; AT2G46680,-0.225135,AT1G29280; AT2G25000; AT5G46350; AT2G46680,-0.223573,AT2G25000; AT1G29280; AT1G80590; AT2G46680,-0.221325,AT1G29280; AT2G03340; AT1G80590; AT2G46680,-0.220887,AT2G46680,-0.080138,3.042927e+05,4.576148e+05,-153322.086793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AT3G16175,AT4G26440; AT1G66550; AT2G34830; AT2G46680,0.109529,AT4G26440; AT1G66550; AT2G21900; AT2G46680,0.105560,AT2G21900; AT1G66550; AT2G34830; AT2G46680,0.102386,AT4G26440; AT1G66550; AT5G56270; AT2G46680,0.102386,AT1G69810; AT4G26440; AT1G66550; AT2G46680,0.098417,AT2G46680,0.015477,3.061545e+01,5.841667e+01,-27.801216
AT4G14550,AT4G24240; AT5G49520; AT2G46680; AT3G04670,0.933884,AT5G07100; AT4G24240; AT5G49520; AT2G46680,0.834800,AT4G24240; AT5G49520; AT2G46680; AT5G26170,0.834782,AT4G24240; AT5G49520; AT5G52830; AT2G46680,0.823723,AT1G69310; AT4G24240; AT5G49520; AT2G46680,0.816677,AT2G46680,0.009967,1.977383e+05,2.388677e+05,-41129.341306
AT2G41480,AT2G23320; AT2G40750; AT4G11070; AT2G46680,-0.361416,AT4G26440; AT4G11070; AT2G46680; AT2G40750,-0.353148,AT2G40750; AT4G11070; AT5G49520; AT2G46680,-0.352477,AT5G01900; AT4G11070; AT2G46680; AT2G40750,-0.350455,AT2G30590; AT4G11070; AT2G46680; AT2G40750,-0.348611,AT2G46680,-0.051201,1.213989e+05,1.874217e+05,-66022.737883
AT1G67340,AT5G15130; AT5G49520; AT1G18860; AT2G46680,0.411299,AT5G15130; AT1G18860; AT2G46680; AT4G26640,0.376395,AT5G15130; AT2G47260; AT1G18860; AT2G46680,0.349534,AT5G15130; AT3G01970; AT1G18860; AT2G46680,0.343478,AT5G15130; AT5G07100; AT1G18860; AT2G46680,0.335425,AT2G46680,0.004958,7.514726e+05,9.390331e+05,-187560.459658
